In [1]:
from forklens.train import Train
from forklens.dataset import ShapeDataset
from torch.utils.data import DataLoader
import torch

import numpy as np
# import matplotlib.pyplot as plt
from astropy.io import fits

Here we train the CNN with 100,000 pairs of mock galaxy and PSF pairs.


In [2]:
# Load data catalog
with fits.open('../../data/csst_catalog_example.fits') as f:
    cat = f[0].data
cut_idx = np.where((cat[:,0]>0.1)&(cat[:,1]<25))[0]
size_cat = cat[cut_idx,0]
mag_cat = cat[cut_idx,1]

### I have actually run the below codes with a slightly different catalog. 
### Therefore the results may look a little bit different.  

In [3]:
# Generate data
def dataframe(num, seed=12345):
    
    num = int(num/2)
    
    rng1 = np.random.RandomState(seed)
    idx = rng1.randint(0,mag_cat.shape[0],size=num)
    Gal_Hlr   = size_cat[idx]
    Gal_Hlr   = np.concatenate((Gal_Hlr, Gal_Hlr), axis=0)
    Gal_Mag   = mag_cat[idx]
    Gal_Mag   = np.concatenate((Gal_Mag, Gal_Mag), axis=0)
    
    rng2 = np.random.RandomState(seed+1)
    Gal_Theta = rng2.random(num*2)*2*np.pi
    Gal_ELL = np.sqrt(rng2.random(num*2))*0.999
    e1 = Gal_ELL*np.sin(2*Gal_Theta)
    e2 = Gal_ELL*np.cos(2*Gal_Theta)

    rng3 = np.random.RandomState(seed+2)
    PSF_randint = rng3.randint(0,10000,size=num)
    PSF_randint = np.concatenate((PSF_randint, PSF_randint), axis=0)
    
    gal_pars = {}
    gal_pars["e1"] = e1
    gal_pars["e2"] = e2
    gal_pars["hlr_disk"] = Gal_Hlr
    gal_pars["mag_i"] = Gal_Mag
    
    psf_pars = {}
    psf_pars['randint'] = PSF_randint
    
    return gal_pars, psf_pars

In the above we generate a input catalog of the training dataset (including both training and validation).

Note two catalogs are generated: one for galaxies and one for PSFs. The catalog for PSFs is simply random index which does not make a difference here, as the simulated PSF image in this code is unchanged for simplicity. 

One may want to rewrite his own code for simulation.py and also to customize dataset.py.  

In [4]:
# Get data loader
nSims = 10000
GalCat, PSFCat = dataframe(nSims, seed=12345)
train_ds = ShapeDataset(GalCat, PSFCat)

In [5]:
torch.cuda.get_device_name()

'Tesla P100-PCIE-16GB'

In [ ]:
world_size = torch.cuda.device_count()
mp.spawn(, args=(world_size, args.save_every, args.total_epochs, args.batch_size), nprocs=world_size)

Some hyper parameters for training are to be modified in ./config.py.

In [6]:
# Train the network
tr = Train()
tr.run(train_ds, show_log=True);

Setting up DDP...
setting device...
initializing process group...
Preparing Data Loader...
Train_dl: 36 Validation_dl: 4


/home/u2/wxs0703/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Begin training ...


/home/u2/wxs0703/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/u2/wxs0703/.local/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x2592 and 1296x512)